<a href="https://colab.research.google.com/github/bankehsaz/Holt-Winters-Trading-Gold/blob/main/Holt_Winters_Model_for_Trading_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import numpy as np
import pandas as pd
import plotly.express as px
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import yfinance as yf

In [2]:
# Get data from yahoo finance
df = yf.download('GLD', start='2022-01-01', end='2024-01-01')
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-07,181.940002,183.000000,180.149994,180.149994,180.149994,9066500
2023-06-08,182.000000,183.039993,181.960007,182.529999,182.529999,6394100
2023-06-09,182.360001,182.789993,181.720001,182.039993,182.039993,4379200
2023-06-12,181.800003,181.940002,181.039993,181.880005,181.880005,4873600
2023-06-13,182.100006,182.429993,180.169998,180.539993,180.539993,5121600


In [3]:
# plot
px.line(df, x=df.index, y='Close')

In [4]:
# Log Transform
df['Log-Close'] = np.log(df['Close'])

In [5]:
# plot
px.line(df, x=df.index, y='Log-Close')

In [6]:
# Split data into train and test
n_test = 30
train = df.iloc[: -n_test]
test = df.iloc[-n_test:]

In [7]:
# Set frequency to 'business day' not working
# df.index.freq = 'B'

In [8]:
# Instantiation
model = ExponentialSmoothing(train['Log-Close'], trend='add', seasonal=None, initialization_method='legacy-heuristic')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [9]:
# Fit the model
res = model.fit()

In [10]:
# Set index for 'Holt-Winters' Column
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

In [11]:
# Calculate Train value and Forecast value for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters-Train'] = res.fittedvalues
df.loc[test_idx, 'Holt-Winters-Test'] = res.forecast(n_test).to_numpy()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [12]:
# Plot
px.line(df, x=df.index, y=['Log-Close', 'Holt-Winters-Train', 'Holt-Winters-Test'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 363 entries, 2022-01-03 to 2023-06-13
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                363 non-null    float64
 1   High                363 non-null    float64
 2   Low                 363 non-null    float64
 3   Close               363 non-null    float64
 4   Adj Close           363 non-null    float64
 5   Volume              363 non-null    int64  
 6   Log-Close           363 non-null    float64
 7   Holt-Winters-Train  333 non-null    float64
 8   Holt-Winters-Test   30 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 28.4 KB
